<a href="https://colab.research.google.com/github/vsaitejaswie/desna/blob/master/tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==1.10.0
!pip install tensorboard==1.10.0
!pip install tensorflow_hub==0.1.1

import pandas as pd

data = pd.read_csv('Consumer_Complaints2.csv', engine = 'python', encoding='utf-8', error_bad_lines=False)
#df.head()
data = data[pd.notnull(data['Consumer complaint narrative'])]
col = ['Product', 'Consumer complaint narrative']
data = data[col]
data.columns = ['product', 'consumer_complaint_narrative']
data.head()

    100% |████████████████████████████████| 58.4MB 631kB/s 
    100% |████████████████████████████████| 573kB 16.5MB/s 
    100% |████████████████████████████████| 12.2MB 4.0MB/s 
    100% |████████████████████████████████| 3.3MB 8.1MB/s 
torchvision 0.2.2.post2 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
thinc 6.12.1 has requirement wrapt<1.11.0,>=1.10.0, but you'll have wrapt 1.11.1 which is incompatible.
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.5 which is incompatible.
pymc3 3.6 has requirement joblib<0.13.0, but you'll have joblib 0.13.2 which is incompatible.
magenta 0.3.19 has requirement tensorflow>=1.12.0, but you'll have tensorflow 1.10.0 which is incompatible.
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.5 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
fastai 1.0.46 has requirement numpy>=1.15, but you'l

    100% |████████████████████████████████| 61kB 3.0MB/s 
  Found existing installation: tensorflow-hub 0.2.0
    Uninstalling tensorflow-hub-0.2.0:
      Successfully uninstalled tensorflow-hub-0.2.0


,product,consumer_complaint_narrative
584,Debt collection,We received a phone call on Tuesday XX/XX/XXXX...
649,"Credit reporting, credit repair services, or o...",Experian informed me that the United State Ban...
659,"Credit reporting, credit repair services, or o...",XXXX informed me that the United State Bankr...
688,"Credit reporting, credit repair services, or o...",Experian didn't properly identify my disputed ...
690,"Credit reporting, credit repair services, or o...",I am writing to dispute a fraudulent charge on...


In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import json
import pickle
import urllib

from sklearn.preprocessing import LabelBinarizer

print(tf.__version__)

1.10.0


In [3]:
pd.set_option('max_colwidth', 500)
data['consumer_complaint_narrative'] = data['consumer_complaint_narrative'].str.lower()
data.head()

,product,consumer_complaint_narrative
584,Debt collection,"we received a phone call on tuesday xx/xx/xxxx from xxxx xxxx xxxx demanding that we call them back about an alleged balance of {$190.00} at the xxxx xxxx xxxx. thursday xx/xx/xxxx we received a written notice in the mail. \n\nhowever, this alleged billing of {$190.00} was disputed in writing every month -- and the lawyer helping us sent 2 letters, most recently on xx/xx/2019, also by email. \n\nwe have proof that the - xxxx xxxx xxxx has no right to pursue any collection actions on us, be..."
649,"Credit reporting, credit repair services, or other personal consumer reports",experian informed me that the united state bankruptcy court provided them an/or reported my information to them in which i wrote the court to find out there procedure for verifying record with the credit reporting agencies. experian still states that its been reported through xxxx xxxx or what they call a third party. please look into this matter and not over this matter. have a blessed day and thank you in advance for your time.
659,"Credit reporting, credit repair services, or other personal consumer reports",xxxx informed me that the united state bankruptcy court provided them an/or reported my information to them in which i wrote the court to find out there procedure for verifying record with the credit reporting agencies. xxxx still states that its been reported through xxxx xxxx or what they call a third party. please look into this matter and not over this matter. have a blessed day and thank you in advance for your time.
688,"Credit reporting, credit repair services, or other personal consumer reports",experian didn't properly identify my disputed update. for account name xxxx xxxx account number xxxx account type credit card i'm requesting proof of liability to this account that is on my credit report and i was only sent an update to the claiming it's mine. under sec. 602. congressional findings and statement of purpose ( a ) accuracy and fairness of credit reporting. the congress makes the following findings : ( 1 ) the banking system is dependent upon fair and accurate credit reportin...
690,"Credit reporting, credit repair services, or other personal consumer reports","i am writing to dispute a fraudulent charge on my account of {$25000.00} i am a victim of identity theft, and i did not make or authorize this charge. i am requesting that the charge be removed, that any finance other charges related to the fraudulent amount be credited, as well and that i receive an accurate statement. this request is made pursuant to the fair credit billing acts amendments to the truth in lending act, 15 u.s.c. 1666-1666b, 12 c.f.r 226.13. see also 12 c.f.r 226.12 ( b ) 1...."


In [4]:

import re
data['consumer_complaint_narrative'] = data['consumer_complaint_narrative'].str.replace('x', '')
data['consumer_complaint_narrative'] = data['consumer_complaint_narrative'].str.replace('{', '')
data['consumer_complaint_narrative'] = data['consumer_complaint_narrative'].str.replace('}', '')
data['consumer_complaint_narrative'] = data['consumer_complaint_narrative'].str.replace('/', '')
data.head()

,product,consumer_complaint_narrative
584,Debt collection,"we received a phone call on tuesday from demanding that we call them back about an alleged balance of $190.00 at the . thursday we received a written notice in the mail. \n\nhowever, this alleged billing of $190.00 was disputed in writing every month -- and the lawyer helping us sent 2 letters, most recently on 2019, also by email. \n\nwe have proof that the - has no right to pursue any collection actions on us, because of a very long and well -documented billing dispute with ..."
649,"Credit reporting, credit repair services, or other personal consumer reports",eperian informed me that the united state bankruptcy court provided them anor reported my information to them in which i wrote the court to find out there procedure for verifying record with the credit reporting agencies. eperian still states that its been reported through or what they call a third party. please look into this matter and not over this matter. have a blessed day and thank you in advance for your time.
659,"Credit reporting, credit repair services, or other personal consumer reports",informed me that the united state bankruptcy court provided them anor reported my information to them in which i wrote the court to find out there procedure for verifying record with the credit reporting agencies. still states that its been reported through or what they call a third party. please look into this matter and not over this matter. have a blessed day and thank you in advance for your time.
688,"Credit reporting, credit repair services, or other personal consumer reports",eperian didn't properly identify my disputed update. for account name account number account type credit card i'm requesting proof of liability to this account that is on my credit report and i was only sent an update to the claiming it's mine. under sec. 602. congressional findings and statement of purpose ( a ) accuracy and fairness of credit reporting. the congress makes the following findings : ( 1 ) the banking system is dependent upon fair and accurate credit reporting. inaccurate...
690,"Credit reporting, credit repair services, or other personal consumer reports","i am writing to dispute a fraudulent charge on my account of $25000.00 i am a victim of identity theft, and i did not make or authorize this charge. i am requesting that the charge be removed, that any finance other charges related to the fraudulent amount be credited, as well and that i receive an accurate statement. this request is made pursuant to the fair credit billing acts amendments to the truth in lending act, 15 u.s.c. 1666-1666b, 12 c.f.r 226.13. see also 12 c.f.r 226.12 ( b ) 1. ..."


In [5]:
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51924 entries, 584 to 133236
Data columns (total 2 columns):
product                         51924 non-null object
consumer_complaint_narrative    51924 non-null object
dtypes: object(2)
memory usage: 1.2+ MB


In [0]:
data_comp=data[['consumer_complaint_narrative']]
data_prod=data[['product']]

In [0]:
train_size = int(len(data_comp) * .999)

train_descriptions = data_comp[:train_size].astype('str')
train_prod = data_prod[:train_size]

test_descriptions = data_comp[train_size:].astype('str')
test_prod =data_prod[train_size:]

In [8]:
print(train_descriptions.shape)
print(test_descriptions.shape)

(51872, 1)
(52, 1)


In [0]:
train_size = int(len(train_descriptions) * .8)

train_desc = train_descriptions[:train_size]
train_pr = train_prod[:train_size]

val_desc = train_descriptions[train_size:]
val_pr =train_prod[train_size:]

In [10]:
print(train_desc.shape)
print(val_desc.shape)

(41497, 1)
(10375, 1)


In [11]:
print(train_pr.shape)
print(val_pr.shape)

(41497, 1)
(10375, 1)


In [12]:
print(train_desc.info())
print(val_desc.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41497 entries, 584 to 110385
Data columns (total 1 columns):
consumer_complaint_narrative    41497 non-null object
dtypes: object(1)
memory usage: 648.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10375 entries, 110390 to 132906
Data columns (total 1 columns):
consumer_complaint_narrative    10375 non-null object
dtypes: object(1)
memory usage: 162.1+ KB
None


In [13]:
from sklearn import preprocessing
encoder = preprocessing.LabelBinarizer()
encoder.fit_transform(train_pr)
train_encoded = encoder.transform(train_pr)
val_encoded = encoder.transform(val_pr)
num_classes = len(encoder.classes_)

# Print all possible products and the label for the first complaint in our training dataset
print(encoder.classes_)
print(train_encoded[0])

['Checking or savings account' 'Credit card or prepaid card'
 'Credit reporting, credit repair services, or other personal consumer reports'
 'Debt collection' 'Money transfer, virtual currency, or money service'
 'Mortgage' 'Payday loan, title loan, or personal loan' 'Student loan'
 'Vehicle loan or lease']
[0 0 0 1 0 0 0 0 0]


In [14]:
description_embeddings = hub.text_embedding_column("descriptions", module_spec="https://tfhub.dev/google/universal-sentence-encoder-large/3", trainable=False)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


In [0]:
multi_label_head = tf.contrib.estimator.multi_label_head(
    num_classes,
    loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE
)

In [16]:
features = {
  "descriptions": np.array(train_desc).astype(np.str)
}
labels = np.array(train_encoded).astype(np.int32)
train_input_fn = tf.estimator.inputs.numpy_input_fn(features, labels, shuffle=True, batch_size=100, num_epochs=10)
estimator = tf.contrib.estimator.DNNEstimator(
    head=multi_label_head,
    hidden_units=[64,10],
    feature_columns=[description_embeddings])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp_dddiu86', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9c75da0278>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
%%timeit
estimator.train(input_fn=train_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp_dddiu86/model.ckpt.
INFO:tensorflow:loss = 0.6974955, step = 1
INFO:tensorflow:Saving checkpoints for 31 into /tmp/tmp_dddiu86/model.ckpt.
INFO:tensorflow:Saving checkpoints for 62 into /tmp/tmp_dddiu86/model.ckpt.
INFO:tensorflow:Saving checkpoints for 93 into /tmp/tmp_dddiu86/model.ckpt.
INFO:tensorflow:global_step/sec: 0.0511202
INFO:tensorflow:loss = 0.29817793, step = 101 (1956.180 sec)
INFO:tensorflow:Saving checkpoints for 124 into /tmp/tmp_dddiu86/model.ckpt.
INFO:tensorflow:Saving checkpoints for 155 into /tmp/tmp_d